In [72]:
import os
import pandas as pd

path = r'REDACTED'
os.chdir(path)
#List of files in folder
file_list = os.listdir(path)

keywords_akkumulerende = ['Akk.', 'Akkumu', 'Akk ', 'Acc'] #Keywords in text for accumulating fund (note Accunia would return True)

def get_institutions(df):
    #Objective: gets the first word in each row of Navn column
    #Returns: list of singular words
    institutions = []
    for value in df.Navn.values:
        if value.split(" ")[0] in ['Investeringsforeningen', 'Kapitalforeningen']:
            institutions.append(value.split(" ")[1])
            continue
        institutions.append(value.split(" ")[0])
    
    return institutions

def foreningsfokus(df):
    #Objective: gets type of fund (from keywords in Navn) in each row of Navn column
    #Returns list of singular words
    foreningsfokus = []
    for value in df.Navn.values:
        if 'æredygtig' in value:
            foreningsfokus.append('Bæredygtig')
        elif 'ustain' in value:
            foreningsfokus.append('Bæredygtig')
        elif 'omentum' in value:
            foreningsfokus.append('Momentum')
        elif 'Small ' in value:
            foreningsfokus.append('Small Cap')
        elif 'Large ' in value:
            foreningsfokus.append('Large Cap')
        elif 'alue' in value:
            foreningsfokus.append('Value')
        elif 'dbytte' in value:
            foreningsfokus.append('Udbytte')
        elif 'udb ' in value.lower():
            foreningsfokus.append('Udbytte')
        elif 'Mid ' in value:
            foreningsfokus.append('Mid Cap')
        #If none of the requirements above are met, we append empty string
        else:
            foreningsfokus.append("")
    
    return foreningsfokus

def akkumulerende(df):
    #Objective: Determine whether a fund is accumulating or not based on the name of the fund
    #Returns: list of Ja or Nej
    
    akkumulerende = []
    for value in df.Navn.values:
        lowercase = value.lower()
        if 'akk.' in lowercase:
            akkumulerende.append('Ja')
        elif 'kkumulerende' in lowercase:
            akkumulerende.append('Ja')
        elif 'akk ' in lowercase:
            akkumulerende.append('Ja')
        elif 'acc' in lowercase:
            #Accunia would return True here, so we take them out with if-else
            if 'accunia' in lowercase:
                akkumulerende.append('Nej')
            else:
                akkumulerende.append('Ja')
        else:
            akkumulerende.append('Nej')
    
    return akkumulerende

def marketshare():
    #Objective: Read 'markedsstatistik-MONTH-YEAR' and calculate market shares
    #Returns: Dataframe with Institution, Bruttoformue for latest month and Marketshare
    path3 = r'REDACTED'
    file = os.listdir(path3)[0]
    os.chdir(path3)
    df_formue = pd.read_excel(file,sheet_name='2.1 Formue (A)',skiprows=2,skipfooter=3)
    fonds_navne = list(df_formue['Kursværdi - mio kr.'][:-1])
    column_name = df_formue.columns[7]
    bruttoformue = list(df_formue[column_name][:-1])
    formue_i_alt = list(df_formue[column_name][-1:])[0]

    market_share = []

    for i, formue in enumerate(bruttoformue):
        #Sætter foreningsgrupper der har udenlandske fonde til marketshare = 0.0, da disse ikke indgår i totalen for danske foreninger
        string_navn = fonds_navne[i]
        if string_navn.split('**')[0] != string_navn:
            market_share.append(0.0)
            continue
        #If value is 'nan' (this check works for some reason)
        elif formue != formue:
            market_share.append(0.0)
            continue
        market_share.append(formue/formue_i_alt)
    
    institutions = []
    for navn in fonds_navne:
        if navn.split(" ")[0] in ['Investeringsforeningen', 'Kapitalforeningen']:
            institutions.append(navn.split(" ")[1])
            continue
        institutions.append(navn.split(" ")[0])
    
    #Manually change I&T's name "Investering" to "I&T" to match the Institution column in df
    indexIT = institutions.index('Investering')
    institutions[indexIT] = 'I&T'
    
    return institutions, bruttoformue, market_share

def omkostninger():
    
    path4 = r'REDACTED'
    file = os.listdir(path4)[0]
    os.chdir(path4)
    df_omkostninger = pd.read_excel(file,skiprows=4)
    del df_omkostninger['Unnamed: 1']
    del df_omkostninger['Dato']
    df_omkostninger = df_omkostninger[(df_omkostninger['Navn'] != "Noteret") | (df_omkostninger['Navn'] != "Noteret")]
    
    navne_liste = list(df_omkostninger.Navn.values)
    omkostninger1 = list(df_omkostninger["Resultatafhængig aflønning"].values)
    omkostninger2 = list(df_omkostninger["Indirekte handels.omk."].values)
    omkostninger3 = list(df_omkostninger["Løbende omk. i %"].values)
    omkostninger4 = list(df_omkostninger["ÅOP"].values)
    
    return navne_liste, omkostninger1, omkostninger2, omkostninger3, omkostninger4 

for file in file_list:
    if len(file_list) != 1:
        print('Make sure the correct file is in the folder - check guide alternatively')
        break
    #We read the Excel file and a specific sheet
    df = pd.read_excel(file, sheet_name = 'Afkast og risikonøgletal', skiprows = 1, skipfooter = 3)
    #Drops missing ISIN values
    df = df[df['ISIN'].notna()]
    #Strips leading spaces from names
    df['Navn'] = df['Navn'].str.lstrip()
    df = df.reset_index()
    del df['index'] #surely we can write this line and the line just above in 1 line?? using inplace = True?
    #Insert new column 'Institution' in column index = 0 using get_institution function
    df.insert(0, 'Institution', get_institutions(df))
    df.insert(4, 'Foreningsfokus', foreningsfokus(df))
    df.insert(5, 'Akkumulerende', akkumulerende(df))
    
    #Marketshare section
    institutions_input, bruttoformue_input, market_share_input = marketshare()
    
    bruttoformue, market_share = [], []

    #Loop over all values in Institution column of df
    for inst in df.Institution.values:
        #If inst is in the df_marketshare Institutions column
        if inst in institutions_input:
            #Set temporary variable index equal to the index so we can retrieve marketshare and bruttoformue
            index = institutions_input.index(inst)
            bruttoformue.append(bruttoformue_input[index])
            market_share.append(market_share_input[index])
            continue
        bruttoformue.append("?")
        market_share.append("?")
    
    #Insert Bruttoformue (mio.DKK) in column index = 6
    df.insert(6, 'Bruttoformue', bruttoformue)
    #Insert Market share in column index = 7
    df.insert(7, 'Marketshare', market_share)
    
    #Omkostninger section
    navne_liste, omkostninger1, omkostninger2, omkostninger3, omkostninger4 = omkostninger()
    
    omkostninger1_final, omkostninger2_final, omkostninger3_final, omkostninger4_final = [], [], [], []
    
    #Loop over all values in Navn column of df
    for navn in df.Navn.values:
        #If navn is in the df_omkostninger Navn column
        if navn in navne_liste:
            #Set temporary variable index equal to the index so we can retrieve omkostninger
            index = navne_liste.index(navn)
            omkostninger1_final.append(omkostninger1[index])
            omkostninger2_final.append(omkostninger2[index])
            omkostninger3_final.append(omkostninger3[index])
            omkostninger4_final.append(omkostninger4[index])
            continue
        omkostninger1_final.append("?")
        omkostninger2_final.append("?")
        omkostninger3_final.append("?")
        omkostninger4_final.append("?")
    
    #Add omkostninger columns to final df
    df["Resultatafhængig aflønning"] = omkostninger1_final
    df["Indirekte handels.omk."] = omkostninger2_final
    df["Løbende omk. i %"] = omkostninger3_final
    df["ÅOP"] = omkostninger4_final

path = r'REDACTED'
os.chdir(path)

filename = 'Masterark til markedsstatistik.xlsx'

writer = pd.ExcelWriter(filename)
df.to_excel(writer, index = False, header = True, sheet_name = "Samlet")
writer.close()
print("Finished")

Finished
